In [ ]:
# Tutorial by www.pylessons.com
# Tutorial written for - Tensorflow 2.3.1

import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import random
import gym
import pylab
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda, Add, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import cv2
import threading
from threading import Thread, Lock
import time
import tensorflow_probability as tfp
from typing import Any, List, Sequence, Tuple


os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

tfd = tfp.distributions

class OurModel(tf.keras.Model):
    def __init__(self, input_shape, action_space):
        super(OurModel, self).__init__()
        
        self.flatten = Flatten()
        self.dense_0 = Dense(512, activation='relu')
        self.dense_1 = Dense(action_space)
        self.dense_2 = Dense(1)
        
    def call(self, X_input):
        X_input = self.flatten(X_input)
        X_input = self.dense_0(X_input)
        action_logit = self.dense_1(X_input)
        value = self.dense_2(X_input)
        
        return action_logit, value


def safe_log(x):
  """Computes a safe logarithm which returns 0 if x is zero."""
  return tf.where(
      tf.math.equal(x, 0),
      tf.zeros_like(x),
      tf.math.log(tf.math.maximum(1e-12, x)))


def take_vector_elements(vectors, indices):
    """
    For a batch of vectors, take a single vector component
    out of each vector.
    Args:
      vectors: a [batch x dims] Tensor.
      indices: an int32 Tensor with `batch` entries.
    Returns:
      A Tensor with `batch` entries, one for each vector.
    """
    return tf.gather_nd(vectors, tf.stack([tf.range(tf.shape(vectors)[0]), indices], axis=1))


huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
sparse_ce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
mse_loss = tf.keras.losses.MeanSquaredError()


class A3CAgent:
    # Actor-Critic Main Optimization Algorithm
    def __init__(self, env_name):
        # Initialization
        # Environment and PPO parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.action_size = self.env.action_space.n
        self.EPISODES, self.episode, self.max_average = 200000, 0, -21.0 # specific for pong
        self.lock = Lock()
        self.lr = 0.000025

        self.ROWS = 80
        self.COLS = 80
        self.REM_STEP = 4

        # Instantiate plot memory
        self.scores, self.episodes, self.average = [], [], []

        self.Save_Path = 'Models'
        self.state_size = (self.REM_STEP, self.ROWS, self.COLS)
        
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.path = '{}_A3C_{}'.format(self.env_name, self.lr)
        self.model_name = os.path.join(self.Save_Path, self.path)

        # Create Actor-Critic network model
        self.ActorCritic = OurModel(input_shape=self.state_size, action_space=self.action_size)
        
        self.learning_rate = 0.0001
        self.optimizer = tf.keras.optimizers.Adam(self.learning_rate, epsilon=1e-05)

    def act(self, state):
        # Use the network to predict the next action to take, using the model
        prediction = self.ActorCritic(state, training=False)
        action = tf.random.categorical(prediction[0], 1).numpy()

        return action[0][0]

    def discount_rewards(self, reward):
        # Compute the gamma-discounted rewards over an episode
        gamma = 0.99    # discount rate
        running_add = 0
        discounted_r = np.zeros_like(reward)
        for i in reversed(range(0, len(reward))):
            if reward[i] != 0: # reset the sum, since this was a game boundary (pong specific!)
                running_add = 0

            running_add = running_add * gamma + reward[i]
            discounted_r[i] = running_add

        discounted_r -= np.mean(discounted_r) # normalizing the result
        discounted_r /= np.std(discounted_r) # divide by standard deviation

        return discounted_r
        
    def get_loss(self, states, actions, discounted_r):
        discounted_r_vstack = np.vstack(discounted_r)
        
        prediction = self.ActorCritic(states, training=True)
        policies = prediction[0]
        values = prediction[1]

        policies_softmax = tf.nn.softmax(policies)

        policies_selected = take_vector_elements(policies, actions)

        #print("discounted_r: ", discounted_r)
        #print("np.stack(values)[:, 0] : ", np.stack(values)[:, 0] )
        advantages = discounted_r - np.stack(values)[:, 0] 

        logits_selected = tf.nn.softmax(policies_selected)
        logits_selected_logs = tf.math.log(logits_selected)

        #print("logits_selected_logs: ", logits_selected_logs)
        #print("advantages: ", advantages)
        actor_loss = -tf.math.reduce_mean(logits_selected_logs * advantages)
        actor_loss = tf.cast(actor_loss, 'float32')

        #critic_loss_ = huber_loss(values, discounted_r)
        discounted_r = tf.cast(discounted_r, 'float32')

        #print("values: ", values)
        critic_loss = tf.reduce_mean(tf.square(values - discounted_r_vstack))

        #entropy_loss = -tf.math.reduce_sum(policies_softmax * tf.math.log(policies_softmax))
        #print("entropy_loss: ", entropy_loss)

        total_loss = actor_loss + 0.5 * critic_loss
            
        return total_loss    
    
    def replay(self, states, actions, rewards):
        # reshape memory to appropriate shape for training
        states = tf.concat(states, 0)
        actions = tf.concat(actions, 0)
        actions = tf.cast(actions, 'int32')
        
        # Compute discounted rewards
        discounted_r = self.discount_rewards(rewards)
        divide_size = 64
        batch_size = states.shape[0]
        epoch_size = batch_size // divide_size
        remain_size = batch_size - epoch_size * divide_size
        #print("batch_size: ", batch_size)
        #print("epoch_size: ", epoch_size)
        #print("remain_size: ", remain_size)
        for e in range(0, epoch_size):
            #print("e: ", e)
            with tf.GradientTape() as tape:
                total_loss = self.get_loss(states[divide_size*e:divide_size*(e+1),:,:,:], 
                                           actions[divide_size*e:divide_size*(e+1)], 
                                           discounted_r[divide_size*e:divide_size*(e+1)])
                
            grads = tape.gradient(total_loss, self.ActorCritic.trainable_variables)
            grads, _ = tf.clip_by_global_norm(grads, 1.0)
            self.optimizer.apply_gradients(zip(grads, self.ActorCritic.trainable_variables))
        '''
        if remain_size != 0:
            #print("remain_size: ", remain_size)
            with tf.GradientTape() as tape:
                total_loss = self.get_loss(states[divide_size*epoch_size:divide_size*epoch_size+remain_size,:,:,:], 
                                           actions[divide_size*epoch_size:divide_size*epoch_size+remain_size], 
                                           discounted_r[divide_size*epoch_size:divide_size*epoch_size+remain_size])
         
            #print("total_loss: ", total_loss)
            #print("")
                
            grads = tape.gradient(total_loss, self.ActorCritic.trainable_variables)
            self.optimizer.apply_gradients(zip(grads, self.ActorCritic.trainable_variables))
        '''
        
    def load(self, model_name):
        self.ActorCritic = load_model(model_name, compile=False)
        #self.Critic = load_model(Critic_name, compile=False)

    def save(self):
        self.ActorCritic.save(self.model_name)
        #self.Critic.save(self.Model_name + '_Critic.h5')

    pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        if str(episode)[-2:] == "00":# much faster than episode % 100
            pylab.plot(self.episodes, self.scores, 'b')
            pylab.plot(self.episodes, self.average, 'r')
            pylab.ylabel('Score', fontsize=18)
            pylab.xlabel('Steps', fontsize=18)
            try:
                pylab.savefig(self.path + ".png")
            except OSError:
                pass

        return self.average[-1]
    
    def imshow(self, image, rem_step=0):
        cv2.imshow(self.model_name + str(rem_step), image[rem_step,...])
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            return

    def GetImage(self, frame, image_memory):
        if image_memory.shape == (1,*self.state_size):
            image_memory = np.squeeze(image_memory)
            
        # croping frame to 80x80 size
        frame_cropped = frame[35:195:2, ::2,:]
        if frame_cropped.shape[0] != self.COLS or frame_cropped.shape[1] != self.ROWS:
            # OpenCV resize function 
            frame_cropped = cv2.resize(frame, (self.COLS, self.ROWS), interpolation=cv2.INTER_CUBIC)
        
        # converting to RGB (numpy way)
        frame_rgb = 0.299*frame_cropped[:,:,0] + 0.587*frame_cropped[:,:,1] + 0.114*frame_cropped[:,:,2]

        # convert everything to black and white (agent will train faster)
        frame_rgb[frame_rgb < 100] = 0
        frame_rgb[frame_rgb >= 100] = 255
        # dividing by 255 we expresses value to 0-1 representation
        new_frame = np.array(frame_rgb).astype(np.float32) / 255.0

        # push our data by 1 frame, similar as deq() function work
        image_memory = np.roll(image_memory, 1, axis = 0)

        # inserting new frame to free space
        image_memory[0,:,:] = new_frame

        return np.expand_dims(image_memory, axis=0)

    def reset(self, env):
        image_memory = np.zeros(self.state_size)
        frame = env.reset()
        for i in range(self.REM_STEP):
            state = self.GetImage(frame, image_memory)
            
        return state
    
    def step(self, action, env, image_memory):
        next_state, reward, done, info = env.step(action)
        next_state = self.GetImage(next_state, image_memory)
        
        return next_state, reward, done, info
    
    def train(self, n_threads):
        self.env.close()
        # Instantiate one environment per thread
        envs = [gym.make(self.env_name) for i in range(n_threads)]

        # Create threads
        threads = [threading.Thread(
                target=self.train_threading,
                daemon=True,
                args=(self,
                    envs[i],
                    i)) for i in range(n_threads)]

        for t in threads:
            time.sleep(2)
            t.start()
            
        for t in threads:
            time.sleep(10)
            t.join()
            
    def train_threading(self, agent, env, thread):
        while self.episode < self.EPISODES:
            # Reset episode
            score, done, SAVING = 0, False, ''
            state = self.reset(env)

            states, actions, rewards = [], [], []
            while not done:
                action = agent.act(state)
                next_state, reward, done, _ = self.step(action, env, state)
                    
                states.append(state)
                actions.append(action)
                rewards.append(reward)

                score += reward
                state = next_state
                    
            self.lock.acquire()
            self.replay(states, actions, rewards)
            self.lock.release()
            
            states, actions, rewards = [], [], []
                    
            # Update episode count
            with self.lock:
                average = self.PlotModel(score, self.episode)
                # saving best models
                if average >= self.max_average:
                    self.max_average = average
                    #self.save()
                    SAVING = "SAVING"
                else:
                    SAVING = ""

                print("episode: {}/{}, thread: {}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, thread, score, average, SAVING))
                if(self.episode < self.EPISODES):
                    self.episode += 1

        env.close()            

    def test(self, Actor_name, Critic_name):
        self.load(Actor_name, Critic_name)
        for e in range(100):
            state = self.reset(self.env)
            done = False
            score = 0
            while not done:
                self.env.render()
                action = np.argmax(self.Actor.predict(state))
                state, reward, done, _ = self.step(action, self.env, state)
                score += reward
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, score))
                    break

        self.env.close()


if __name__ == "__main__":
    env_name = 'PongDeterministic-v4'
    #env_name = 'Pong-v0'
    agent = A3CAgent(env_name)
    
    #agent.run() # use as A2C
    agent.train(n_threads=1) # use as A3C
    #agent.test('Models/Pong-v0_A3C_2.5e-05_Actor.h5', '')

episode: 0/200000, thread: 0, score: -21.0, average: -21.00 SAVING
episode: 1/200000, thread: 0, score: -20.0, average: -20.50 SAVING
episode: 2/200000, thread: 0, score: -20.0, average: -20.33 SAVING
episode: 3/200000, thread: 0, score: -21.0, average: -20.50 
episode: 4/200000, thread: 0, score: -20.0, average: -20.40 
episode: 5/200000, thread: 0, score: -21.0, average: -20.50 
episode: 6/200000, thread: 0, score: -20.0, average: -20.43 
episode: 7/200000, thread: 0, score: -19.0, average: -20.25 SAVING
episode: 8/200000, thread: 0, score: -20.0, average: -20.22 SAVING
episode: 9/200000, thread: 0, score: -21.0, average: -20.30 
episode: 10/200000, thread: 0, score: -21.0, average: -20.36 
episode: 11/200000, thread: 0, score: -20.0, average: -20.33 
episode: 12/200000, thread: 0, score: -20.0, average: -20.31 
episode: 13/200000, thread: 0, score: -19.0, average: -20.21 SAVING
episode: 14/200000, thread: 0, score: -21.0, average: -20.27 
episode: 15/200000, thread: 0, score: -21.0,